# Extracción de datos
En este notebook se tratará la extracción de datos de sitios web como Reddit o Github, mediante APIs expresamente hechas por el sitio, o funciones de Python que servirán para realizar las peticiones.

Para empezar, se cargarán los ajustes definidos en el apartado anterior junto con la importación de algunas librerías básicas necesarias para comenzar. 

In [6]:
import sys, os

#Carga del archivo setup.py
%run -i setup.py

#Carga del archivo settings.py
#%run "$BASE_DIR/settings.py"
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

You are working on a local system.
Files will be searched relative to "..".


## Extracción de datos con la librería *request*

Para este primer ejemplo se va a hacer uso de la librería *request* que se incluye con la instalación de python. Esta librería es el método más básico para acceder y extraer información a través de una API, pero al mismo tiempo es una herramienta muy potente.

Véamos a continuación como se realiza una petición GET en la que se tratará de listar todos los repositorios que cuenten con unas características  determinadas. En este caso, por ejemplo, vamos a listar todos los repositorios relacionados con el protocolo Zigbee que, al no usar el token de autenticación de un usuario registrado, nos devolverá aquellos repositorios que sean públicos, si se quieren listar también aquellos privados, hay que proporcionar el token de acceso.

In [7]:
import requests
import json

response = requests.get('https://api.github.com/search/repositories',
    params={'q': 'zigbee'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'})

print(response.status_code)
#print(response.json())

200


Para poder entender de forma más clara la lista de repositorios obtenida, vamos a dar formato Markdown al archivo json y se mostrarán los 5 primeros resultados de la respuesta obtenida:

In [8]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

for item in response.json()['items'][:5]:
    printmd('**' + item['name'] + '**' + ': repository ' +
            item['text_matches'][0]['property'] + ' - \"*' +
            item['text_matches'][0]['fragment'] + '*\" matched with ' + '**' +
            item['text_matches'][0]['matches'][0]['text'] + '**')

**zigbee2mqtt.io**: repository name - "*zigbee2mqtt.io*" matched with **zigbee2mqtt**

**zigbee2mqtt**: repository description - "*Zigbee 🐝 to MQTT bridge 🌉, get rid of your proprietary Zigbee bridges 🔨*" matched with **Zigbee**

**zigbee**: repository name - "*zigbee*" matched with **zigbee**

**zigbee-herdsman-converters**: repository description - "*Collection of device converters to be used with zigbee-herdsman*" matched with **zigbee**

**hassio-zigbee2mqtt**: repository name - "*hassio-zigbee2mqtt*" matched with **zigbee2mqtt**

### Listar comentarios del apartado *Issues* de un repositorio
Es posible listar todos los comentarios del apartado Issues de un repositorio si así se especifica en la petición el nombre del repositorio y el dueño del mismo:

In [9]:
response = requests.get(
    'https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments')
print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 200
Number of comments 30


Se observa que únicamente ha recopilado 30 comentarios, esto es porque la API de github limita el número de elementos para cada respuesta. Si mostramos por pantalla los enlaces de la respuesta obtendremos el número de páginas contenidas en la respuesta.

In [10]:
response.links

{'next': {'url': 'https://api.github.com/repositories/302841413/issues/comments?page=2',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/302841413/issues/comments?page=90',
  'rel': 'last'}}

### Paginación
Se usa la paginación para limitar el número de elementos en una que se devolverán tras una petición.

Al mostrar los links de *response* se ve que proporciona el enlace a la siguiente página a la respuesta y a la última del total.

Para obtener todos los resultados se debe definir una función que llame a la siguiente página, y así hasta que se hayan procesado todas de forma recursiva.

Del mismo modo, importaremos *Pandas* a nuestro código para transformar los datos obtenidos en un Data Frame.

En el siguiente ejemplo se muestra lo anteriormente explicado además de mostras por pantalla el número de filas que se desee del Data Frame:

***Por alguna razón a veces me coge bien todos los comentarios y a veces no***

In [11]:
import pandas as pd

def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json


out = get_all_pages(
    "https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments",
    params={
        #Como parámetros indicamos la fecha desde la cuál queremos obtener los resultados,
        #el orden (de creación) y la dirección
        'since': '2020-07-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept': 'application/vnd.github.v3+json'})

#Los resultados obtenidos los transformamos en un Data Frame para su posterior análisis
df = pd.DataFrame(out)
pd.set_option('display.max_colwidth', 1)

# #Muestra por pantalla el total de resultados
print(df['body'].count())

# #Muestra por pantalla algún ejemplo de los resultados obtenidos
df[['id', 'created_at', 'body']].sample(10, random_state=42)

1770


,id,created_at,body
974,1494575704,2023-04-03T15:53:42Z,"Not all devices support reporting properly, if reporting is supported you have to use optimistic=true."
275,2303377137,2024-08-22T01:10:39Z,still present
411,2118999280,2024-05-18T20:47:05Z,"I confirm, same problem.\r\nI'm not sure if this is because of zigbee2mqtt, but after launching the add-on, the Raspberry Pi actually began to freeze. No useful logs.\r\nHardware: Raspberry Pi 3 Model B+, Sonoff ZBDongle-E.\r\n\r\nI tried:\r\n- perform a completely clean installation of HASS OS without restoring a backup;\r\n- restore backups with earlier versions of zigbee2mqtt and HA where everything worked before;\r\n- change HASS OS to Raspberry Pi OS Lite;\r\n- change Sonoff ZBDongle-E firmware;\r\n- change Raspberry Pi.\r\nNo result.\r\n\r\nStrange facts:\r\n- if you start the Raspberry Pi and insert ZBDongle-E after loading, then zigbee2mqtt starts;\r\n- if you install ZHA, it works fine."
962,1500969933,2023-04-08T20:16:06Z,I think I fixed it by disabling bluetooth in the config.txt\r\n\r\ndtoverlay=disable-bt
518,1981707341,2024-03-06T20:15:49Z,"an uninstall and reinstall of zigbee2mqtt fixed this issue, still don't know what caused it"
1252,1322257341,2022-11-21T15:42:34Z,"Not long before one of the switches got loose, I updated some of them through the OTA tab. Could this have any effect? Even now, I have switches with different refresh times working. And is it possible to somehow reset the firmware or update them forcibly? I also have Xiaomi Gateway 3, but these switches are also not connected to it.\r\n![image](https://user-images.githubusercontent.com/70881046/203096268-5202e218-e54d-4e1b-902b-500a57fa5306.png)\r\n"
1085,1413537895,2023-02-02T10:48:33Z,"Hello, \r\nI've got the same issue on a Raspberry Pi 4b with a ConBee II, running on HAOS.\r\nI removed the DeConz Add-on and the ZHA integration, reinstalled Z2M and Mosquitto. I made sure the ConBee II is plugged on a USB 2.0 port, with an extension cable. \r\nI modified multiple times the config, directly in the FileEditor but the error keeps happening and apparently blocks the connection to the MQTT server.\r\nI don't know what to do...\r\n\r\nHere is the config : \r\n\r\n```\r\ndata_path: /config/zigbee2mqtt\r\nsocat:\r\n enabled: false\r\n master: pty,raw,echo=0,link=/tmp/ttyZ2M,mode=777\r\n slave: tcp-listen:8485,keepalive,nodelay,reuseaddr,keepidle=1,keepintvl=1,keepcnt=5\r\n options: ""-d -d""\r\n log: false\r\nmqtt:\r\n base_topic: zigbee2mqtt\r\n server: mqtt://192.68.2.83:1883\r\n user: USER\r\n password: PASSWORD\r\nserial:\r\n adapter: deconz\r\n port: /dev/ttyACM0\r\nzigbee_herdsman_debug: true\r\n\r\n```\r\n"
344,2202529650,2024-07-02T09:24:20Z,I dont have a group with those high IDs. I have up to 11 groups
1050,1429712939,2023-02-14T13:01:40Z,Hello\r\nNo solution at this time with and update....
1457,1207664454,2022-08-08T04:55:10Z,Keep alive


### Límites de las APIs
Las APIs tienen un límite a la hora de devolver los resultados, por ejemplo, se pueden haber obtenido 400 comentarios, pero si se accede al sitio del que estos han sido extraídos, cabe la posibilidad de que el total sea ampliamente mayor.

Por esto, se va a definir una función que impida sobrecargar el servidor al que se le realizan las peticiones, disminuyendo la velocidad entre una petición y la siguiente y asegurarnos de que toda la información que se ha solicitado sea descargada correctamente.

In [12]:
from datetime import datetime
import time

def handle_rate_limits(response):
    now = datetime.now()
    reset_time = datetime.fromtimestamp(
        #X_RateLimit indica cuántas peticiones se puden realizar por unidad de tiempo
        int(response.headers['X-RateLimit-Reset']))
    
    #X-RateLimit-Remaining indica cuántas peticiones pueden aún hacerse
    #sin superar el límite establecido
    remaining_requests = response.headers['X-RateLimit-Remaining']
    remaining_time = (reset_time - now).total_seconds()
    intervals = remaining_time / (1.0 + int(remaining_requests))

    print('Esperando por ', intervals)
    time.sleep(intervals)

    return True

La librería *requests* no contempla una función que permita reintentar la petición en caso de error, aún con esto, se puede implementar gracias a la librería *HTTPAdapter*.

In [13]:
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

retry = Retry(
    #5 reintentos
    total=5,
    #Códigos de erros los cuáles si se reciben, se reintentará
    status_forcelist=[500, 503, 504],
    #Retraso entre reintentos después del segundo intento
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("https://", retry_adapter)

response = http.get('https://api.github.com/search/repositories',
                    params={'q': 'zigbee'})

for item in response.json()['items'][:5]:
    print(item['name'])

zigbee2mqtt.io
zigbee2mqtt
zigbee
zigbee-herdsman-converters
hassio-zigbee2mqtt


Juntando las funciones definidas en el apartado de paginación junto con las definidas en este último, quedaría como resultado algo así:

In [16]:
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

retry = Retry(
    #5 reintentos
    total=5,
    #Códigos de erros los cuáles si se reciben, se reintentará
    status_forcelist=[500, 503, 504],
    #Retraso entre reintentos después del segundo intento
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("https://", retry_adapter)

def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json


out = get_all_pages(
    "https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments",
    params={
        #Como parámetros indicamos la fecha desde la cuál queremos obtener los resultados,
        #el orden (de creación) y la dirección
        'since': '2020-07-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept': 'application/vnd.github.v3+json'})

#Los resultados obtenidos los transformamos en un Data Frame para su posterior análisis
df = pd.DataFrame(out)
pd.set_option('display.max_colwidth', 1)

# #Muestra por pantalla el total de resultados
#print(df['body'].count())

# #Muestra por pantalla algún ejemplo de los resultados obtenidos
df[['id', 'created_at', 'body']].sample(5, random_state=42)

KeyError: "None of [Index(['id', 'created_at', 'body'], dtype='object')] are in the [columns]"